#### Load libraries, data and params

In [15]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
#for projections
from pyproj import Transformer

In [ ]:
#inputs----------
dsto = xr.open_dataset('') # for bathymetry
dsb0 = xr.open_dataset('')
dst = xr.open_dataset('') # dir with particle tracks (for getting initial positions)

#data for rt
PATH_DISC = ""
#save folder
PATH_SAVE_DISC = ""

#to change - adjust - depends on the amount of data which is to process:
START_DATE = 0
start_date_str = ""
units = ""
YEARS = 1

In [ ]:
#parameters---
#volume_correction=True
npa_per_dep=0 #number of particles per deployment
m2=int(0) #period in seconds
dx=0;dy=0 #particle initial grid resolution in km
#nobs=0   #tracks will have maximum Tr of 116M2 ~ 60days (instead of 90 days)
per_min=0 #min % per point of release where at least per_min of the particles leave the DWS

#### Useful functions

In [6]:
########################################################
#create regular grid for statistics (RT, ET, ...) (to use contourf)
def create_grid(x0,y0):
    xmin=x0.min();xmax=x0.max();ymin=y0.min();ymax=y0.max()
    xgrid=np.arange(xmin-dx*1e3,xmax+2*dx*1e3,dx*1e3)
    ygrid=np.arange(ymin-dy*1e3,ymax+2*dy*1e3,dy*1e3)
    xgrid0,ygrid0=np.meshgrid(xgrid,ygrid)
    return xgrid0,ygrid0


########################################################
#gridding data with nearest
def gridding_nearest(var,x0,y0,xgrid,ygrid):
    vargrid=xgrid.flatten()*np.nan
    tree = KDTree(np.c_[xgrid.flatten(),ygrid.flatten()]) #points in the new extended grid
    _,ij = tree.query(np.c_[x0,y0],k=1) #get index for every x0,y0 to put values in the new grid
    vargrid[ij]=var
    vargrid=np.reshape(vargrid,xgrid.shape)
    return vargrid


########################################################
def create_cmap(numcolors=11,colors=['blue','white','red'],name='create_cmap'):
    """
    Create a custom diverging colormap
    Default is blue to white to red with 11 colors. Colors can be specified
    in any way understandable by matplotlib.colors.ColorConverter.to_rgb()
    """
    from matplotlib.colors import LinearSegmentedColormap
    cmap = LinearSegmentedColormap.from_list(N=numcolors,colors=colors,name=name)
    return cmap

########################################################
def get_index_valid_points_groupby_bins(npa_per_dep,num_deploys,dat,idp_xy,per_min,typee='out'):
    #this is indentical to the above rolling mean, when subsampling=window_length
    #this only work for gather data using interval (days) and interval_min_dat (days)
    #for gatter data in months use resample
    #the advantage of this function in contrast to rolling is that it can be used with data that dont have a dt multiple or divisor of m2 
 

    #1)------------------------------------
    #total amount of particles---
    npa_total=npa_per_dep*num_deploys 
    #
    #particles stuck---
    npa_stuck_xy=(idp_xy==0).sum(dim='timedep')
    per_stuck_xy=npa_stuck_xy/num_deploys*100
    npa_stuck_total=npa_stuck_xy.sum().values
    per_stuck_total=np.round(npa_stuck_total/npa_total*100,2)
    #
    #particles no stuck but always inside the DWS---
    #here there are issues with particles deployed in the bank of the mainland
    #they are no stuck but remains almost all the time in the same positions
    npa_in_xy=(idp_xy==1).sum(dim='timedep')
    per_in_xy=npa_in_xy/num_deploys*100
    npa_in_total=npa_in_xy.sum().values
    per_in_total=np.round(npa_in_total/npa_total*100,2)
    #
    #particles that leave at least once the DWS---
    npa_out_xy=(idp_xy==2).sum(dim='timedep')
    #npa_out_xy=(idp_xy!=1).sum(dim='timedep')
    per_out_xy=npa_out_xy/num_deploys*100
    npa_out_total=npa_out_xy.sum().values
    per_out_total=np.round(npa_out_total/npa_total*100,2)

    print("particles per deployment =", npa_per_dep)
    print("num deployments = ",num_deploys)
    print(f"stuck = {npa_stuck_total}({per_stuck_total}%)")
    print(f"always inside = {npa_in_total}({per_in_total}%)")
    print(f"out DWS = {npa_out_total}({per_out_total}%)")
    print(f"total particles = {npa_total}")
 
    #2)------------------------------------
    #recompute always leaving, considering only points where % > per_min---
    #so we are:
    #- considering particles that leave at lease once the DWS, which are deployed from regions
    #  where they represent more than per_min% of the deployments per grid (25290)
    #- helping to avoid problematic regions that tend to have stuck and always inside particles
    #this part give particle positions mostly outside banks--
    idp_out_opt_xy=(per_out_xy>per_min)
    npa_out_opt_xy=npa_out_xy.where(idp_out_opt_xy)
    per_out_opt_xy=npa_out_opt_xy/num_deploys*100
    npa_out_opt_total=npa_out_opt_xy.sum().values
    per_out_opt_total=np.round(npa_out_opt_total/npa_total*100,2)
    print(f"out DWS (> {per_min}% per x,y) = {npa_out_opt_total}({per_out_opt_total}%)")
    #
    #get a boolean index when: (idp_xy==2) or (idp_xy>0) and (per_out_xy % > per_min). It is similar to idp_xy but only with True and False---
    #this part only modifies above computations if we also consider particles "always inside DWS"
    if typee=='out': ind_out_opt_xy=(idp_xy==2)*idp_out_opt_xy #consider only "out" particles for points where % > per_min
    if typee=='inout': ind_out_opt_xy=(idp_xy>0)*idp_out_opt_xy #consider "always in" and "out" particles for points where % > per_min
    per_out_opt_total=np.round(ind_out_opt_xy.sum().values/npa_total*100,2) 
    print(f"{typee} DWS (> {per_min}% per x,y) = {ind_out_opt_xy.sum().values}({per_out_opt_total}%)")
    print(f"num points {typee} DWS = ",idp_out_opt_xy.sum().values)
        
    return ind_out_opt_xy

#### Calculate rt

In [ ]:
xct0=dsto.xc.min().values/1e3; yct0=dsto.yc.min().values/1e3 #=(0,0)
#create mask for islands from topo file
mask=dsto.bathymetry.copy(); mask=xr.where(np.isfinite(mask),1,0) #mask ocean=1, land=0


#initial particle positions---
x0=dst.x.values; y0=dst.y.values
#getting particle position in a grid---
xgrid,ygrid=create_grid(x0,y0)

bdr_dws0=dsb0.bdr_dws.values #points that define DWS

In [ ]:
#initialize variables
start_date = np.datetime64(start_date_str)
date = START_DATE
rt_xy = None
idp_xy = None
time = []

rt_15mean = None

#loops over years and months
for i in range(0,YEARS):
    date = date + i
    for j in ['01','02','03','04','05','06','07','08','09','10','11','12']: #months
        file = PATH_DISC + "\\" + str(date) + "\\" + str(date) + j + ".nc"
            
        print("file: ", file)
        #read data : time, rt and idp
        rt = xr.open_dataset(file)
        time.extend(rt.timedep.values.astype('datetime64[ns]'))
        if rt_xy is None:
            rt_xy = rt.rt
            idp_xy = rt.idp
        else:
            rt_xy = xr.concat([rt_xy, rt.rt], dim='timedep')
            idp_xy = xr.concat([idp_xy, rt.idp], dim='timedep')
        rt.close()

        #for each 15 days interval
        while time[-1] >= start_date + np.timedelta64(15, 'D'):

            end_date = start_date + np.timedelta64(15, "D")
            print("\nstart_date: ", start_date)
            print("end_date: ", end_date)
            rt_15 = rt_xy.sel(timedep=slice(start_date, end_date))
            idp_15 = idp_xy.sel(timedep=slice(start_date, end_date))

            num_deploys=idp_15.shape[0]

            #find indexes of "good" particles---
            ind_out_opt_xy = get_index_valid_points_groupby_bins(npa_per_dep,num_deploys,rt_15,idp_15,per_min,typee='out')
            rt_15out=rt_15.where(ind_out_opt_xy)

            #convert to numpy array
            rt_15out = np.array(rt_15out)
            #mask nan values and compute mean
            masked_rt = np.ma.masked_array(rt_15out, np.isnan(rt_15out))
            rt_15mean_ = np.mean(masked_rt, axis=0).filled(np.nan)

            #griding using 400mx400m squares around the center position of the deployment---
            #particles were released skiping every one 200mx200m grid cell, so above grid is just for plotting to cover the full domain and avoid empty spaces 
            rt_15mean_ = gridding_nearest(rt_15mean_, x0, y0, xgrid, ygrid)

            rt_15mean_ = rt_15mean_.reshape(1, rt_15mean_.shape[0], rt_15mean_.shape[1])
            
            #add the data to list
            if rt_15mean is None:
                rt_15mean = rt_15mean_
            else:
                rt_15mean = np.concatenate([rt_15mean, rt_15mean_], axis=0)

            #remove data which has been used
            rt_xy = rt_xy.sel(timedep=slice(end_date, None))
            idp_xy = idp_xy.sel(timedep=slice(end_date, None))
            time = [t for t in time if t >= end_date]
            
            #update start_date
            start_date = end_date

#### For rotations

In [34]:
#define the coordinate transformations----------
#1)
#from epgs:28992(DWS) to epgs:4326(LatLon with WGS84 datum used by GPS and Google Earth)
proj = Transformer.from_crs('epsg:28992','epsg:4326',always_xy=True)
#2)
#from epgs:4326(LatLon with WGS84) to epgs:28992(DWS) 
inproj = Transformer.from_crs('epsg:4326','epsg:28992',always_xy=True)
#inproj_old=Proj("EPSG:28992") #old method (has errors 10-20m when contrast with the rotated coords)

#lon,lat to 28992(DWS)-projection--------------------

#bathymetry--------
xct=dsto.lonc.values;  yct=dsto.latc.values #lon,lat units
xctp,yctp,z = inproj.transform(xct,yct,xct*0.)
xctp=(xctp)/1e3; yctp=(yctp)/1e3 
#first projected point to correct the coordinates of model local meter units
xctp0=xctp[0,0]; yctp0=yctp[0,0]

#matrix rotation -17degrees-----
ang=-17*np.pi/180
angs=np.ones((2,2))
angs[0,0]=np.cos(ang); angs[0,1]=np.sin(ang)
angs[1,0]=-np.sin(ang); angs[1,1]=np.cos(ang)


#local meter model units to 28992(DWS)-projection and lon-lat--------------

#bathymetry----
#original topo points in meter
xct2,yct2=np.meshgrid(dsto.xc.values,dsto.yc.values)
xy=np.array([xct2.flatten(),yct2.flatten()]).T
#rotate
xyp=np.matmul(angs,xy.T).T/1e3
xyp0=xyp[0,:] #the first point in the bathy data in local meter units=0,0

#contour of DWS------
#rotate
bdr_dws0p=np.matmul(angs,bdr_dws0.T).T/1e3
#correct model units:
#1)substact the first model local point of the topo file, but give tha same as xyp0=[0,0]
#2)use the first projected point of the case (lon,lat model units to meter)
bdr_dws0p=bdr_dws0p-xyp0 
bdr_dws0p[:,0]=bdr_dws0p[:,0]+xctp0; bdr_dws0p[:,1]=bdr_dws0p[:,1]+yctp0
#get coordinates in lon-lat units (WGS84 ) 
bdr_dws0_lon, bdr_dws0_lat, z = proj.transform(bdr_dws0p[:,0]*1e3,bdr_dws0p[:,1]*1e3, bdr_dws0p[:,1]*0.)

#regular grid of statistics------
xy=np.array([xgrid.flatten(),ygrid.flatten()]).T
#rotate
xyp=np.matmul(angs,xy.T).T/1e3
#correct model units:
#1)substact the first model local point of the topo file, but give tha same as xyp0=[0,0]
#2)use the first projected point of the case (lon,lat model units to meter)
xyp=xyp-xyp0 
xyp[:,0]=xyp[:,0]+xctp0; xyp[:,1]=xyp[:,1]+yctp0 
xyp=np.reshape(xyp,(xgrid.shape[0],xgrid.shape[1],2))
xgridp=xyp[...,0]; ygridp=xyp[...,1] #km

### Plot

In [ ]:
#Tr---
fig=plt.figure(figsize=(7.32,1.95),dpi=600) #120
gs = fig.add_gridspec(nrows=1, ncols=2, hspace=.2, wspace=.1)                   
cmap=create_cmap(numcolors=2,colors=['w','w'])
cmap.set_bad(color='wheat') #nan=gray

val=rt_15mean[0]; title="15day mean rt"; lab="a)"
#
#plot mean---
gss=gs[0,0]
ax = fig.add_subplot(gss)
ax.pcolormesh(xctp[:,:-1],yctp[:,:-1],dsto.bathymetry[:,:-1],vmin=-2,vmax=32,cmap=cmap,shading='auto',rasterized=True,zorder=0) #avoid artificial eastern wall 
cs=ax.contourf(xgridp,ygridp,val,levels=np.arange(0,30.2,.2),cmap="jet",extend='max',zorder=0) #"Spectral_r"
ax.contour(xctp[:,:-1],yctp[:,:-1],mask[:,:-1],levels=[1],linewidths=.5,colors='k',zorder=1) #avoid artificial eastern wall
ax.plot(bdr_dws0p[:,0],bdr_dws0p[:,1],ls='-',color='k',lw=.8,markersize=0,zorder=2)
ax.contour(xctp[:,:-1],yctp[:,:-1],dsto.bathymetry[:,:-1],levels=[5],linewidths=.3,colors='dimgrey',zorder=3)
#
ax.grid(linewidth=0.4,ls=':')
yticks=np.arange(540,640,20); ax.set_yticks(yticks);ax.set_yticklabels(yticks-540);
xticks=np.arange(100,240,20); ax.set_xticks(xticks);ax.set_xticklabels(xticks-100);
ax.axis('equal'); ax.axis([xticks[0],xticks[-1],yticks[0],yticks[-1]]);
ax.tick_params(direction="in")
ax.set_title(f"{title}")
ax.text(102,540+84,lab)
ax.set_xlabel('Easting (km)')
if i==0: ax.set_ylabel('Northing (km)');
if i==1: ax.set_yticklabels("")


cbar=fig.colorbar(cs,ax=ax,ticks=np.arange(0,40,10),aspect=10,pad=0.03);
cbar.set_label(label="Residence time (days)",rotation=90)
cbar.ax.tick_params(labelsize=10)

### Saving

In [68]:
from netCDF4 import date2num
from datetime import datetime, timedelta

start_date = datetime.strptime(start_date_str+" 00:00:00", "%Y-%m-%d %H:%M:%S")
interval = timedelta(days=15)
n_periods = len(rt_15mean)
middle_times = [start_date + interval * i + interval / 2 for i in range(n_periods)]

time = np.array(
        date2num(
            middle_times, units=units, calendar="standard"
        ),
        dtype=np.float64,
    )

In [ ]:
dsout = xr.Dataset()
dsout["Rt_mean"]=(("time","yrc","xrc"),rt_15mean)
dsout["Rt_mean"].attrs["long_name"] = "mean 15-day residence time"
dsout["Rt_mean"].attrs["description"] = "NaN values are for points outside DWS and the ones removed from the analysis"
dsout["Rt_mean"].attrs['units'] = 'day'

dsout.coords["xr"]=(("yrc","xrc"),xgridp)
dsout.coords["xr"].attrs["long_name"] = "x-position for the residence times"
dsout.coords["xr"].attrs['units'] = 'km'
dsout.coords["yr"]=(("yrc","xrc"),ygridp)
dsout.coords["yr"].attrs["long_name"] = "y-position for the residence times"
dsout.coords["yr"].attrs['units'] = 'km'
#
dsout.coords["xh"]=(("yc","xc"),xctp)
dsout.coords["yh"]=(("yc","xc"),yctp)
dsout.coords["xh"].attrs["long_name"] = "x-position for the bathymetry"
dsout.coords["xh"].attrs['units'] = 'km'
dsout.coords["yh"].attrs["long_name"] = "y-position for the bathymetry"
dsout.coords["yh"].attrs['units'] = 'km'
#
dsout.coords["time"]=time
dsout.coords["time"].attrs["long_name"] = "time"
dsout.coords["time"].attrs["units"] = units
#
dsout["h"]=(("yc","xc"),dsto.bathymetry.values)
dsout["h"].attrs["long_name"] = "bathymetry"
dsout["h"].attrs['units'] = 'm'
#
dsout["bdr_dws"]=(("np_dws","xy_dws"),bdr_dws0p)
dsout["bdr_dws"].attrs["long_name"] = 'xy coordinates of the boundary of the DWS'
dsout["bdr_dws"].attrs['units'] = 'km'


dsout.to_netcdf("rt_and_rotated_coord.nc")